In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在笔记本中对BigQuery数据进行交互式探索性分析

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/exploratory_data_analysis/explore_data_in_bigquery_with_workbench.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/exploratory_data_analysis/explore_data_in_bigquery_with_workbench.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub中查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/exploratory_data_analysis/explore_data_in_bigquery_with_workbench.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

这个笔记本是为在BigQuery中有数据并希望在互动环境中进行探索性数据分析以从数据中收集见解的数据分析师和数据科学家编写的。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/managed/visualize-data-bigquery)和了解更多关于[BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml)。

### 目标

在本教程中，您将了解如何在 Jupyter notebook 环境中探索和从 BigQuery 数据中获得洞察。

本教程使用以下 Google Cloud 数据分析和 ML 服务：

- BigQuery
- Vertex AI
- Vertex AI Workbench（可选，也可以在其他地方运行，比如 [Colab](https://colab.sandbox.google.com/)）

执行的步骤包括：

- 使用 Python 和 SQL 查询 BigQuery 中的公共数据
- 使用 BigQuery INFORMATION_SCHEMA 探索数据集
- 创建交互式元素以帮助探索数据的有趣部分
- 进行一些探索性相关性和时间序列分析
- 在笔记本中创建静态和交互式输出（数据表格和图表）
- 将一些输出保存到 Cloud Storage

### 数据集

该数据集来自于谷歌商品商店的混淆[Google Analytics 4 (GA4)数据](https://developers.google.com/analytics/bigquery/web-ecommerce-demo-dataset)，可以在[BigQuery上公开访问](https://console.cloud.google.com/bigquery?project=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&p=bigquery-public-data&page=dataset)。

成本

本教程使用Google Cloud的计费组件：

* BigQuery
* Vertex AI
* Vertex AI Workbench（可选）
* Cloud Storage

了解 [BigQuery Pricing](https://cloud.google.com/bigquery/pricing), [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Vertex AI Workbench pricing](https://cloud.google.com/vertex-ai/pricing#notebooks), 和 [Cloud Storage pricing](https://cloud.google.com/storage/pricing)以根据您预计的使用情况生成成本估算。

### 安装额外的软件包

安装并/或升级以下软件包，以执行此笔记本所需的操作。

In [ ]:
# Install/upgrade Vertex AI ("aiplatform") library
! pip3 install --upgrade google-cloud-aiplatform  -q

# Install/upgrade BigQuery storage library
! pip3 install --upgrade google-cloud-bigquery-storage  -q

# Install ipywidgets to use interactive widgets in notebooks
! pip3 install --upgrade ipywidgets  -q

# Install itables to use interactive tables in notebooks
! pip3 install --upgrade itables  -q

# Install kaleido to save plotly images as static files
! pip3 install --upgrade kaleido  -q

# Install plotly to create interactive plots
! pip3 install --upgrade plotly  -q

### 仅限Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 在开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量.
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
- 不需要做任何操作，因为您已经得到验证。

**2. 本地JupyterLab实例**，请取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
- 在这里查看所有身份验证选项：[Google Cloud Platform Jupyter Notebook身份验证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶来存储诸如数据集之类的中间工件。

In [ ]:
BUCKET_NAME = "your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

UUID

如果您在一个实时教程会话中，您可能正在使用共享测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个UUID，并将其附加到本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

导入库并配置一些选项。

In [ ]:
import os

# Import library to enable some interactive widgets
import ipywidgets as widgets
# Import options from itables separately
import itables.options as itable_opts
# Import basic Python data science libraries
import numpy as np
import pandas as pd
# Import plotly express for quick plots
import plotly.express as px
# Import AI Platform library (as Vertex AI) and BigQuery library
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
# Import Exceptions library to help with error catching
from google.cloud.exceptions import BadRequest
# Import IPython display utilities
from IPython.display import clear_output
# Import particular interactive capabilities
from ipywidgets import interact
# Import itables for interactive tables
from itables import show

In [ ]:
# Configure some options related to interactive tables
itable_opts.maxBytes = 1e9
itable_opts.maxColumns = 50

itable_opts.order = []
itable_opts.column_filters = "header"

### 为使用准备好 Vertex AI 和 BigQuery

在这个笔记本中为您的项目初始化 Vertex AI SDK for Python，并设置 BigQuery 以与您的项目一起使用。

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建BigQuery客户端。

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

为向BigQuery发送查询创建一个辅助函数。您将在接下来的部分中多次使用此函数。

In [ ]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def bq_query(sql, show_job_id=False):
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    try:
        job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)

        bq_client.query(sql, job_config=job_config)

    except BadRequest as err:
        print(err)
        return

    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running, then get & return data frame
    df = client_result.result().to_dataframe()

    if show_job_id:
        print(f"Finished job_id: {job_id}")

    return df

## 在BigQuery中探索Google Analytics 4（GA4）数据结构
在这一部分中，您将查看示例Google Analytics 4（GA4）数据的结构：列字段、行数、涵盖的日期等。

### 查看表数据结构

BigQuery中的样本GA4数据是按日期[分区](https://cloud.google.com/bigquery/docs/partitioned-tables)的，这使得更容易管理和查询数据。有多种方法可以查询特定日期、一组日期（例如特定月份）以及所有日期。在此笔记本中，您主要执行的是后者。

您可以使用[`%%bigquery` Python魔术命令](https://googleapis.dev/python/bigquery/latest/magics.html)在笔记本单元格中直接执行SQL查询，如下单元格所示。

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *

FROM
  /* Use '*' at end of table name to allow querying over multiple date 
     partitions of same overall table */
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

LIMIT 100

上面的结果表明，这张表格有点宽，具有一些[嵌套列](https://cloud.google.com/bigquery/docs/nested-repeated)和半结构化数据。一些关键列-事件日期，事件时间戳和事件名称-出现在左侧。

### 按日期查看事件数量
接下来，您会得到一个表格，显示数据中每个事件日期的事件数量。

在%%bigquery魔法旁边的"event_dates"可以将查询结果保存到具有该名称的pandas DataFrame中。

In [ ]:
%%bigquery event_dates --project $PROJECT_ID 
SELECT
  event_date,
  COUNT(*) AS num_events

FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

GROUP BY
  event_date

ORDER BY
  event_date

将查询结果通过在其自己的单元格中引用结果DataFrame进行打印。

In [ ]:
event_dates

您看到的数据中有三个月的日期，从2020年11月到2021年1月，在每天大约有数万个事件（至少对于默认输出中可见的日期）。

查看数据中事件的总数

In [ ]:
sum(event_dates.num_events)

这份数据中有大约430万个事件行。鉴于此以及部分列的相对深度，最好在BigQuery中处理它，并将相关结果提取到Python中（而不是将整个表提取到Python中）。

通过使用 INFORMATION_SCHEMA 了解更多关于数据结构
在这个部分，您可以使用 BigQuery 的 INFORMATION_SCHEMA 的各种功能来更深入了解 GA4 数据。

获取每个表分区的一些信息
以下是一个查询，使用"\_\_TABLES__"功能在示例GA4数据中获取每个分区表的一些信息（行数、大小等）。

In [ ]:
%%bigquery --project $PROJECT_ID 
SELECT 
  table_id,

  (CASE
     WHEN type = 1 THEN 'Table'
     WHEN type = 2 THEN 'View'
     ELSE NULL
     END) AS type,

  row_count,
  ROUND(size_bytes / POW(2, 20), 1) AS size_MB,
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) AS last_modified_time 

FROM
  /* __TABLES__ functionality to look at all tables in given dataset */
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce`.__TABLES__

ORDER BY
  table_id  

您可以再次看到，这里有92个表，每个表对应于跨度中的每个日期，每个表中有几十MB的数据。

#### 获取每个表列的信息
接下来，您可以使用BigQuery INFORMATION_SCHEMA中的[COLUMNS view](https://cloud.google.com/bigquery/docs/information-schema-columns)来获取GA4示例数据表中每个列的一行。

In [ ]:
# Get max event date to use for query below
max_event_date = max(event_dates["event_date"])

# Get information on columns in GA4 data (using only 1 date table)
info_schema_columns_sql = f"""
    SELECT *
    
    FROM
      `bigquery-public-data.ga4_obfuscated_sample_ecommerce`.
      INFORMATION_SCHEMA.COLUMNS
      
    WHERE
      table_name = 'events_{max_event_date}'
    """

info_schema_columns = bq_query(info_schema_columns_sql)

info_schema_columns

您可以看到表格有23列，包括字符串、整数、浮点数，以及几个嵌套或重复的字段。

获取每个（嵌套的）字段的交互式信息表
要更深入地探查上表中的嵌套字段（因为它们有各种有用的字段），您可以使用[COLUMN_FIELD_PATHS视图](https://cloud.google.com/bigquery/docs/information-schema-column-field-paths)。

In [ ]:
info_schema_column_field_paths_sql = f"""
  SELECT
    column_name,
    field_path,
    data_type
  
  FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce`.
    INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    
  WHERE
    table_name = 'events_{max_event_date}'
  """

info_schema_column_field_paths = bq_query(info_schema_column_field_paths_sql)

在这种情况下，使用[itables包](https://mwouts.github.io/itables/quick_start.html)创建一个交互式表格来显示查询结果。这样可以让您直接在笔记本单元格输出中的表格中探索、过滤或排序数据。

In [ ]:
show(info_schema_column_field_paths)

上表中有108行，显示了不同层次的层次结构。使用列过滤器，您可以查看包含时间字段、用户信息和地理（位置）数据的列字段。您还可以查看设备字段，这些字段可以是分类的、数值的或其他数据类型的。

## 探索不同类别的事件计数
在本节中，您可以使用绘图库——特别是 [matplotlib](https://matplotlib.org/) 和 [Plotly Express](https://plotly.com/python/plotly-express/)——生成图形输出，帮助您了解样本 GA4 数据中各个领域的事件分布情况。

### 探索按类型的事件数量
首先，您可以从BigQuery获取整个数据中按类型（即“事件名称”）划分的事件数量，并生成结果的静态条形图。

In [ ]:
event_name_counts_sql = """
    SELECT
      event_name,
      COUNT(*) AS num_events

    FROM
      `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

    GROUP BY
      event_name

    ORDER BY
      num_events DESC
    """

event_name_counts = bq_query(event_name_counts_sql)

# matplotlib used to create basic bar plot
event_name_counts_bar_plot = event_name_counts.plot(
    kind="bar", x="event_name", y="num_events"
)

event_name_counts_bar_plot

从柱状图中可以看出，页面浏览量和用户互动是两种最常见的事件类型，其他类型的事件则出现了大幅下降。

您可以使用Plotly Express在交互式柱状图中显示相同事件类型分布数据。

In [ ]:
event_name_counts_interactive_bar_plot = px.bar(
    event_name_counts,
    x="event_name",
    y="num_events",
    title="Number of Events by event_name",
)

event_name_counts_interactive_bar_plot.show()

条形图显示与上一个相同的信息，还具有交互功能。您可以将鼠标悬停在每个条形上以查看实际事件数，或放大图表的各个部分（例如，查看较不常见事件的数字在较小比例上的比较）。

### 用于泛化事件计数条形图创建的函数
假设您喜欢先前的交互式事件计数条形图，并希望查看数据中与设备、用户特征或其他字段相关的其他分类变量。一种很好的泛化方法是创建一个函数，该函数从BigQuery获取任何单个变量的事件计数数据，然后创建一个类似上面的交互式条形图。运行以下单元格以创建这样一个函数。

In [ ]:
def get_event_counts_by_categorical_var_bar_plot(
    bq_var_name, output_var_name, min_events=500
):

    # SQL code "template" to get event counts for specific variable in data
    event_counts_sql = f"""
        SELECT
          IFNULL({bq_var_name}, 'NULL') AS {output_var_name},
          COUNT(*) AS num_events

        FROM
          `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

        GROUP BY
          {bq_var_name}
        
        /* Filter to only return values of variable that have some min. # of
           events - default is 500, can set to 0 to do no filtering */
        HAVING
          num_events >= {min_events}
        
        ORDER BY
          num_events DESC
        """

    event_counts = bq_query(event_counts_sql)

    # Take results from SQL query and turn into bar plot for given variable
    event_counts_interactive_bar_plot = px.bar(
        event_counts,
        x=output_var_name,
        y="num_events",
        title=(
            f"<b>Number of Events by {output_var_name}<br></b>"
            + f"Min. {min_events} Events for {output_var_name}"
        ),
    )

    return event_counts_interactive_bar_plot

### 按国家、操作系统和设备语言分类的事件数量
在这里，您可以使用上面创建的函数来按照数据中的三个不同分类变量（国家、操作系统和设备语言）获取事件计数。这些显示在同一个输出单元格的三个单独选项卡中，使用[ipywidgets包的选项卡功能](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Tabs)。

In [ ]:
# Create table of categorical variables (BQ and output names) to loop over
categorical_vars_df = pd.DataFrame.from_records(
    data=[
        ("geo.country", "country"),
        ("device.operating_system", "operating_system"),
        ("device.language", "device_language"),
    ],
    columns=["bq_var_name", "output_var_name"],
)

# Initialize tab outputs list to be appended to
tab_outputs = []

# Loop over categorical variables data, generating 1 bar plot per tab
for index, row in categorical_vars_df.iterrows():
    this_tab = widgets.Output()

    with this_tab:
        get_event_counts_by_categorical_var_bar_plot(
            row["bq_var_name"], row["output_var_name"]
        ).show()

    # Append tab output for this tab to list of outputs
    tab_outputs = tab_outputs + [this_tab]

# Create output with all tab outputs together
output = widgets.Tab(tab_outputs)

# Loop over categorical variables again to set output tab titles
for index, row in categorical_vars_df.iterrows():
    output.set_title(index, row["output_var_name"])

display(output)

通过点击选项卡，您可以看到：

- 美国是活动最多的国家，其次是印度和英国。
- “Web”是迄今为止最常见的设备操作系统，iOS 和 Windows 在第二位。
- 设备语言字段中最常见的值实际上是 NULL，即该字段中没有数据。在任何进一步的分析或建模中，这一点非常重要。您可能想要了解为什么存在缺失值、缺失语言的用户/活动类型更频繁还是不频繁，以及这在未来数据中可能如何改变或不改变。

### 互动：选择类别，获取相应事件计数的条形图
在前一部分的基础上，您甚至可以进一步实现互动性，允许从数据中存在的一组类别中进行选择，然后动态地为所选的变量创建图表。在下面，您将创建一个设备属性的交互式列表，并设置事件最小数量的滑块，然后将这些参数传递给函数，以使用这些参数创建互动事件计数图。每次更改其中一个参数，代码都会重新运行以相应地创建新的图表。

（注意：您必须在"活动"内核中运行此笔记本才能在下面的单元格中使用交互菜单。如果您只是查看之前生成的输出，它将无法工作。）

In [ ]:
# Filter to categorical fields that are nested under device (to use as options)
device_cat_vars = info_schema_column_field_paths[
    (info_schema_column_field_paths["column_name"] == "device")
    & (info_schema_column_field_paths["data_type"] == "STRING")
].reset_index(drop=True)

# Some string manipulation to make device-related variable names
device_cat_vars["var_name"] = [
    ("device_" + field_path.split(".")[-1])
    for field_path in device_cat_vars["field_path"]
]


# Interact section that creates a function that responds to user input
@interact
def show_device_field_event_counts(
    # Widget to select single variable to look at event counts for
    device_field=widgets.Select(
        options=device_cat_vars["var_name"].tolist(),
        value="device_category",
        description="Device Attributes",
        continuous_update=False,
    ),
    # Slider to pick min. # of events required to return data for given category
    min_num_events=widgets.IntSlider(
        value=500,
        min=0,
        max=1e5,
        step=100,
        description="Min Num Events",
        continuous_update=False,
    ),
):

    clear_output()

    # Get BQ variable name for selected device field using data frame
    bq_var_name = device_cat_vars.loc[
        device_cat_vars["var_name"] == device_field, "field_path"
    ].item()

    # Use function to generate event counts bar plot for chosen device field
    event_count_plot = get_event_counts_by_categorical_var_bar_plot(
        bq_var_name, device_field, min_num_events
    )

    return event_count_plot.show()

请随意在上面的单元格中选择不同的小部件组合，查看生成的图表。例如，如果您保留默认设置（设备类别和500事件最小值），您会发现桌面是最常见的设备类别，其次是移动设备，然后是平板电脑，占所有事件的比例要小得多。

此处交互性的一个好处是您可以通过相对快速的点按选项查看许多图表，而无需编写重复的代码或预先确定要绘制的内容。

通过国家和操作系统的热图生成事件分布。基于之前步骤的分析，你也可以查询和绘制两个分类变量的组合的事件分布。你之前分别查看了国家和操作系统(OS)的个别事件分布。在这种情况下，你关注*国家和操作系统的组合*。除了每个国家-OS组合中事件的原始数量外，你还计算该数量占该国所有事件数量的百分比，以查看某些国家在特定操作系统上的活动更多/更少。

In [ ]:
country_os_counts_sql = """
    SELECT
      geo.country AS country,
      device.operating_system AS os,
      
      COUNT(*) AS num_events,
      
      /* Window aggregate function to country total events (across all OS) */
      SUM(COUNT(*)) OVER (PARTITION BY geo.country)
        AS tot_events_in_country,
      
      /* Look at # of events in country-os combination as % of all events in
        that country,  */
      SAFE_DIVIDE(
        COUNT(*),
        SUM(COUNT(*)) OVER (PARTITION BY geo.country)
        ) * 100 AS pct_events_in_country

    FROM
      `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

    GROUP BY
      geo.country, device.operating_system
    
    QUALIFY
      tot_events_in_country >= 50000
    
    ORDER BY
      num_events DESC
    """

country_os_counts = bq_query(country_os_counts_sql)

country_os_counts

对于这种两类别联合分布数据，一个有用的可视化方法是热力图。在下面，您可以使用Plotly Express的热力图功能创建一个可视化，显示每个国家中事件占所有设备操作系统的百分比。

In [ ]:
# Pivot country-OS data to get in form necessary for Plotly Express heat map
country_os_counts_matrix = pd.pivot_table(
    country_os_counts,
    values="pct_events_in_country",
    index=["os"],
    columns=["country"],
    aggfunc=np.sum,
)

# Create heat map % of events in each country (columns) by device OS (rows)
country_os_counts_heatmap = px.imshow(
    country_os_counts_matrix,
    labels=dict(x="Country", y="Device OS", color="% Events in Country on This OS"),
    title="Percent of Events in Each Country by Device OS",
)

country_os_counts_heatmap.show()

从观察颜色和悬停在一些数字上来看，可以清楚地看出跨设备操作系统的事件分布在各个国家并没有太大差异。在这些国家中，Web 是主要的操作系统，占每个国家分析的事件的55-65%。在一些国家中还有一些其他系统稍微比例更高（比如法国的Windows，新加坡的iOS），但你很可能会从热力图中发现各国之间看起来相对一致。

在这一部分，您将查看用户级别的事件数据。所有对应于相同user_pseudo_id的事件被分组在一起，然后您对用户的活动进行分析（而不是像之前的部分那样对单个事件进行分析）。

计算和探索用户级别的综合指标
首先，您可以使用BigQuery在用户级别生成一些有趣的综合数据：页面浏览量、用户参与度和总购买价值（销售额）。

In [ ]:
%%bigquery user_aggregates --project $PROJECT_ID
  SELECT
    user_pseudo_id,
    
    /* Use array functionality to get most recent language by user
       (in case of >1 language per user, which is possible) */
    IFNULL((ARRAY_AGG(device.language ORDER BY event_timestamp DESC)
      [SAFE_ORDINAL(1)]), 'NULL') 
      AS language,
    
    SUM(IF(event_name = 'page_view', 1, 0))
      AS num_page_views,
    
    SUM(IF(event_name = 'user_engagement', 1, 0))
      AS num_user_engagements,
    
    SUM(IF(event_name = 'purchase', event_value_in_usd, 0))
      AS tot_purchase_value_usd

  FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

  GROUP BY
    user_pseudo_id
    
  ORDER BY
    tot_purchase_value_usd DESC, num_page_views DESC

In [ ]:
user_aggregates

通过查看用户聚合输出，您可以看到在样本GA4数据中有大约270K个不同的用户，其中最大的消费者从在线商店购买的商品总价超过1400美元。

你现在可以利用[Plotly的散点图功能](https://plotly.com/python/line-and-scatter/)进一步探索这些用户数据，查看互动次数与页面浏览量之间的关系。下面的代码可以做到这一点，同时根据总购买价值调整点的大小，并根据用户设备语言对其进行着色。

In [ ]:
user_page_views_engagements_scatter_plot = px.scatter(
    user_aggregates,
    x="num_page_views",
    y="num_user_engagements",
    size="tot_purchase_value_usd",
    color="language",
    title=(
        "<b>User-Level Engagements vs Page Views</b><br>"
        + "(Each Point is 1 User, Sized by Total Purchase Value)"
    ),
)

user_page_views_engagements_scatter_plot.show()

通过玩弄这个互动图表，你可以看到很多内容，包括：

* 用户页面浏览量和参与度之间存在极强的正相关关系，这从逻辑上是合理的。
* 页面浏览量和参与度较高的用户通常似乎拥有更高的总购买价值，尽管你可以看到有一些用户在左下角花费更多，而在右上角相对花费较少。
* 除了少数例外，最活跃的用户（页面浏览量和参与度均很高）通常是那些语言设置为"en-us"或者（更常见的情况）没有已知设备语言的用户。你之前已经知道这两个值是所有事件中language字段最常见的，所以这可能与之一致，也算是毫不奇怪的。

### 检查用户级指标之间的相关性
您在上面的图表中看到了用户页面浏览量、参与度和购买价值之间的一些相关性。接下来，您将计算跨用户之间这些量之间的实际相关值。

In [ ]:
user_aggregates.corr()

用户页面浏览量与互动之间的相关性非常高（0.98），如预期，而每一个相关性与购买价值之间的相关性为正值，但要低得多（0.38）。

超越总体相关性，您可能希望了解这些关系在您数据的不同分片中是否相似 - 例如用户人口统计数据。在下面的查询中，您可以使用BigQuery内置的相关性功能，计算每个页面浏览量和购买价值的相关性，但这次是按国家分组的。

In [ ]:
%%bigquery user_purchase_value_correlations_by_country --project $PROJECT_ID 
/* Use structure of user_aggregates query from above in WITH clause, as first 
   step to aggregate data to user level */
WITH
user_aggregates AS
(
  SELECT
    user_pseudo_id,

    /* Use array functionality to get most recent country by user
       (in case of >1 country per user, which is possible) */
    IFNULL((ARRAY_AGG(geo.country ORDER BY event_timestamp DESC)
      [SAFE_ORDINAL(1)]), 'Unknown') 
      AS country,
    
    SUM(IF(event_name = 'page_view', 1, 0))
      AS num_page_views,
    
    SUM(IF(event_name = 'user_engagement', 1, 0))
      AS num_user_engagements,
    
    SUM(IF(event_name = 'purchase', event_value_in_usd, 0))
      AS tot_purchase_value_usd

  FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

  GROUP BY
    user_pseudo_id
)

SELECT
  country,
  COUNT(*) AS num_users,
  CORR(num_page_views, tot_purchase_value_usd)
    AS corr_page_views_purchase_value,
  
  CORR(num_user_engagements, tot_purchase_value_usd)
    AS corr_engagements_purchase_value

FROM
  user_aggregates

GROUP BY
  country

/* Only return countries w/ 500+ users (for higher sample size) */
HAVING
  num_users >= 500    
    
ORDER BY
  num_users DESC

通过从BigQuery读取的数据，现在可以在交互式表格中显示它（带有一些格式化，以避免不得不阅读非常长的十进制列）。

In [ ]:
with pd.option_context("display.float_format", "{:,.3f}".format):
    show(user_purchase_value_correlations_by_country)

从在两个方向上对相关性列排序交互表开始，我们可以看到用户活动指标和购买价值之间的相关性在不同国家之间存在差异。一些国家如希腊和哥伦比亚的相关性大于0.6，而其他国家如比利时和意大利的相关性小于0.25。这表明了客户网站活动与销售之间的转化在世界各地有所不同 - 这是一个具有潜力进行深入研究并对如何对购买行为建模产生影响的话题。

## 通过日期和时间购买的时间序列分析
另一个常见的探索性数据分析模式是查看一定时间内的特定指标。有一个完整的领域 - 以及专门用于时间序列预测的[强大的谷歌云工具](https://cloud.google.com/learn/what-is-time-series) - 但在这种情况下，你将专注于通过日期和/或时间字段聚合的样本GA4数据的更基础的探索。

### 创建模板查询，按日期和时间聚合购买价值

您想要分析购买数据，首先按日期聚合，然后还要按每天的小时进行聚合。为了避免重复编写代码，一种方法是编写类似下面的SQL查询模板，该模板按日期聚合并提供输入对应于小时字段的文本的选项（该字段也添加到聚合的分组中）。

在此示例中，您只想关注美国的数据，以简化世界各地日期意义不同的问题。为了实现这一点，下面的代码仅筛选美国事件，并将所有事件移动到它们在美国太平洋时间的日期/时间上（[Google Analytics数据中的事件时间戳字段是在UTC时间](https://support.google.com/analytics/answer/7029846)），以匹配该国通常的白天/黑夜时间。

In [ ]:
purchases_by_date_hour_sql_template = """
    WITH
    USDateAggregates AS
    (
      SELECT
        /* Use event_timestamp and move things to date in US (Pacific Time) */
        EXTRACT(DATE FROM TIMESTAMP_MICROS(event_timestamp) 
          AT TIME ZONE "America/Los_Angeles") AS event_date,

        /* Add template field for hour of day, can use to aggregrate or split */
        {hour_of_day_sql} AS hour_of_day,        

        COUNT(*) AS num_purchases,
        SUM(event_value_in_usd) AS total_purchase_value

      FROM 
        `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`

      WHERE
        /* Filter to US data only */
        geo.country = 'United States' AND
        /* Look only at purchases with non-NULL values */
        event_name = 'purchase' AND
        event_value_in_usd IS NOT NULL

      GROUP BY
        event_date, hour_of_day
    )

    SELECT
      event_date,
      hour_of_day,

      /* Extract various date-related fields from event date */
      EXTRACT(YEAR FROM event_date) AS year,
      EXTRACT(MONTH FROM event_date) AS month,
      EXTRACT(DAY FROM event_date) AS day,

      EXTRACT(WEEK FROM event_date) AS week_of_year_num,
      EXTRACT(ISOWEEK FROM event_date) AS iso_week,

      EXTRACT(DAYOFWEEK FROM event_date) AS day_of_week_num,
      FORMAT_DATE("%A", event_date) AS day_of_week,

      EXTRACT(DAYOFYEAR FROM event_date) AS day_of_year,

      num_purchases,
      total_purchase_value,

      SAFE_DIVIDE(
        total_purchase_value,
        num_purchases
        ) AS avg_purchase_value

    FROM
      USDateAggregates

    ORDER BY
      event_date, hour_of_day
    """
("")

### 按日期查看总购买价值

首先，您查看谷歌分析数据中三个月的日期购买总计。

In [ ]:
# Run query with 'ALL' (string) for hour of day to aggregate all hours together
purchases_by_date = bq_query(
    purchases_by_date_hour_sql_template.format(hour_of_day_sql="'ALL'")
)

您可以使用[Plotly的线状图功能](https://plotly.com/python/line-charts/)绘制每日购买销售额的交互式时间序列图。

In [ ]:
total_purchase_value_by_date_plot = px.line(
    purchases_by_date,
    x="event_date",
    y="total_purchase_value",
    # Add day of week to hover data to help read off interactive plot
    hover_data=["event_date", "day_of_week", "total_purchase_value"],
    title="<b>United States Total Purchase Value (USD) by Date<b>",
)

total_purchase_value_by_date_plot.show()

从这个图表中可以看到一些事情：

* 美国零售业的销售趋势一般情况下是在11月销售量呈现正增长趋势，圣诞节前购物季节销售量较高（直至12月中旬），然后在年底假期前销售量急剧下降，这种下降趋势大多持续到1月份。
* 每天的总销售额存在相当大的波动。在极端情况下，该时期销售额最高的一天是[“网购星期一”](https://en.wikipedia.org/wiki/Cyber_Monday)，而销售额最低的一天（在有完整数据的日子中）是圣诞节 - 这两点都相对不令人惊讶。
* 一周中有一些与销售量相关的影响，例如星期一相对较高，而周末相对较低。

### 查看每周的每天和每天的每个小时的总购买价值
接下来，您可以进一步收集每天和每小时的总购买数据 - 这是比上文更细的聚合级别 - 并使用热力图来研究常见的购买模式。

In [ ]:
# Run query with string corresponding to hour of day (in US Pacific Time)
purchases_by_date_hour = bq_query(
    purchases_by_date_hour_sql_template.format(
        hour_of_day_sql=(
            "EXTRACT(HOUR FROM TIMESTAMP_MICROS(event_timestamp)"
            + "AT TIME ZONE 'America/Los_Angeles')"
        )
    )
)

要以所需的“星期时间线”形式获取热力图，你需要进一步聚合到星期几和当天时间段，并将数据格式转换成正确的形式。热力图中的每个单元格代表了在三个月的数据中给定星期的特定时间所产生的总销售额（例如，左上角代表了太平洋时间12至1点之间所有星期日的销售额）。

In [ ]:
# Add day of week field combining number and name, to help with sorting on plot
purchases_by_date_hour["day_of_week_combo"] = (
    purchases_by_date_hour["day_of_week_num"].astype("str")
    + " - "
    + purchases_by_date_hour["day_of_week"]
)

# Pivot to day-of-week/hour-of-day grid, summing purchase value while doing so
purchases_by_dayofweek_hourofday_matrix = pd.pivot_table(
    purchases_by_date_hour,
    values="total_purchase_value",
    index=["day_of_week_combo"],
    columns=["hour_of_day"],
    aggfunc=np.sum,
)

purchases_by_dayofweek_hourofday_heatmap = px.imshow(
    purchases_by_dayofweek_hourofday_matrix,
    labels=dict(x="Hour of Day", y="Day of Week", color="Total Purchase Value"),
    title=("<b>United States Total Purchase Value (USD) by Day & Hour (PDT)<b>"),
)

purchases_by_dayofweek_hourofday_heatmap.show()

从这个图表中你可以看到一些事情：

*   在不同的日子和时段，总销售额存在显著的变化。在极端情况下，有些日子/时段的销售额超过了$\$$2,000，而另一些则不到$\$$100。
*   正如你在每日销售图中看到的，周六和周日的销售额相对较低（可能晚上周日有例外），而周一相对较高（每天不同时段存在不同的高峰）。大部分国家深夜时段（x轴上的0-5）通常在大部分日子销售较慢，但也存在一些异常值。

在上面的图表中，一些展示的情况可能与基于时区的预期并不完全吻合，因此可能值得检查是否存在一些异常值（高销售额）可能会偏离某些日子/时段的数据。

有时候您可能希望您的一些代码输出（表格、图形等）能够持久保存在笔记本外部。这可能是为了在您完成笔记本时保留一些结果，或者通过其他媒介与各种合作者分享。

在这一部分，您将采取一些先前生成的输出，并以三种不同的方式保存在笔记本外部：
* 将表格保存为CSV文件
* 将交互式图表保存为HTML文件
* 将交互式图表保存为（静态）PNG文件

首先，您可以将用户级别的国家相关性表保存为CSV格式。最初，这将会将CSV文件保存在本地笔记本环境中。

In [ ]:
user_country_correlations_output_name = "user_level_correlations_by_country.csv"

user_purchase_value_correlations_by_country.to_csv(
    user_country_correlations_output_name
)

接下来，您可以使用[Plotly的HTML导出功能](https://plotly.com/python/interactive-html-export/)将按名称计算的事件计数的交互式条形图保存为HTML文件。

In [ ]:
event_name_counts_output_name = "event_name_counts_interactive_bar_plot.html"

event_name_counts_interactive_bar_plot.write_html(event_name_counts_output_name)

虽然互动图可以很强大，但通常会有情况需要将互动图转换为静态图以适应特定限制或简化读者阅读内容的情况。在这种情况下，您可以使用[Plotly的静态图像导出功能](https://plotly.com/python/static-image-export/)保存按日和小时购买的热力图为静态PNG文件。这会在“底层”使用[kaleido库](https://github.com/plotly/Kaleido)来将互动图转换为静态图。

In [ ]:
purchases_day_hour_output_name = "purchases_by_dayofweek_hourofday_heatmap.png"

purchases_by_dayofweek_hourofday_heatmap.write_image(purchases_day_hour_output_name)

最后，您在本地笔记本环境中取出三个不同的输出文件，并将它们复制到您指定的云存储桶中。

In [ ]:
output_file_names = [
    user_country_correlations_output_name,
    event_name_counts_output_name,
    purchases_day_hour_output_name,
]

for output_file_name in output_file_names:
    ! gsutil cp $output_file_name $BUCKET_URI

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目]。

否则，如果您愿意，您可以删除您在此教程中创建的Cloud Storage存储桶：

In [ ]:
# Delete Cloud Storage bucket that was created
if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI